In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import numpy as np
import torch.nn as nn
import re
from torch.utils.data import DataLoader

In [2]:
# Normal dataset
df = pd.read_csv("smallDomainDataNotEmbedded.csv")

In [3]:
# Updating values for training_data
training_data = df[df['split'] == 'train']
#training_data = training_data.drop(training_data.query('toxicity==0').sample(frac=.85).index)

# Getting test_data
test_data = df[df['split'] == 'test']

# Getting validation_data
validation_data = df[df['split'] == 'val']
#validation_data = validation_data.drop(validation_data.query('toxicity==0').sample(frac=.85).index)

In [4]:
df

,comment_text,split,toxicity,male,female,LGBTQ,christian,muslim,other_religion,black,white
0,does this fool ever work for Canadians. I can...,train,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Bottom line is all religious extremist of all ...,test,1,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
2,"Here we go again, those damned Mormons.",val,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Prove it Ms Khalid. Launch the exact same mot...,train,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,So long Europe. So many people are afraid of b...,test,1,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2855,These traitors (read the Constitution!) pose m...,val,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2856,OMG. Your rant reeks of desperation. So now yo...,val,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2857,Hey all you White Boys ... F)(K you and your e...,val,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2858,Learn history kid! Your one of the many reason...,val,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
# Creating data loaders
X_train = np.array(training_data['comment_text'].values.tolist())
Y_train = np.array(training_data['toxicity'].values.tolist())

X_test = np.array(test_data['comment_text'].values.tolist())
Y_test = np.array(test_data['toxicity'].values.tolist())

X_val = np.array(validation_data['comment_text'].values.tolist())
Y_val = np.array(validation_data['toxicity'].values.tolist())

In [43]:
print(len(X_train))
print(len(X_test))
print(len(X_train))
print(len(X_val))
print(Y_val)

1420
720
1420
720
[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 

In [ ]:
tokenized_training = []
for i in range(len(X_train)):
    tokenized_training.append(tokenizer.tokenize(X_train[i]))

In [7]:
lengths = []
for i in range(len(tokenized_training)):
    lengths.append(len(tokenized_training[i]))
lengths = np.array(lengths)    
print(np.mean(lengths))
print(np.median(lengths))
lengths = np.sort(lengths)

90.534176182708
72.0


In [7]:
def CleanText(text):
    text = text.lower() #Turn all text entries into lower-case
    text = re.sub(r'''(https?:\/\/www\.|https?:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,3}[-a-zA-Z0-9()@:%_\+.~#?&\//=<>]*''', "<URL>", text)
    #Replace URL with tag
    text = re.sub(r'''[0-9]+[/\-.]+[0-9]+[/\-.]+[0-9]+''', "<DATE>", text) #Replace dates with tag
    text = re.sub(r'''[a-z0-9._%+-]+\@[a-z0-9.-]+[a-z0-9]\.[a-z]{1,}''', "<EMAIL>", text)
    text = re.sub(r'''[0-9]+''', "<NUM>", text) #Replace numbers with tag
    text = re.sub(r'''[.|,|!|?|\'|\''|\"|\n|\t|\-|\(|\)]''', '', text)
    text = re.sub(r'''^\s+|\s+$''', '', text) #Remove whitespaces at the end and start of string
    text = re.sub(r'''[ ][ ]+|_''', " ", text) #Remove multiple whitespace
    return text


def BuildSentenceMatrix(dataset, embedding_dimension, features):
    
    converted_dataset = []
    
    with torch.no_grad():
        for i, text in enumerate(dataset):
            text = CleanText(text)
            inputs = tokenizer(text, return_tensors="pt")
            inputs = inputs.to('cuda')           
            
            outputs = model(**inputs)
            outputs = outputs[0].squeeze()
        
            if len(outputs) < embedding_dimension:
                zeroVectors = torch.zeros((embedding_dimension - len(outputs), features))
                zeroVectors = zeroVectors.to('cuda')
                outputs = torch.cat((outputs, zeroVectors))
            elif len(outputs) > embedding_dimension:
                outputs = outputs[:embedding_dimension]
            
            if i%20 == 0:
                print(i)
            converted_dataset.append(outputs)
    
    return converted_dataset
    


In [6]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
#model = AutoModel.from_pretrained("bert-base-uncased")
#model.eval()
#model.to('cuda')

from transformers import BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
model.to('cuda')


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [7]:
prep_training = []
prep_test = []
prep_val = []

for i, text in enumerate(X_train):
    prep_training.append([torch.tensor(tokenizer.encode(text, add_special_tokens=True, max_length=100, truncation=True, padding="max_length")), Y_train[i]])

for i, text in enumerate(X_test):
    prep_test.append([torch.tensor(tokenizer.encode(text, add_special_tokens=True, max_length=100, truncation=True, padding="max_length")), Y_test[i]])

for i, text in enumerate(X_val):
    prep_val.append([torch.tensor(tokenizer.encode(text, add_special_tokens=True, max_length=100, truncation=True, padding="max_length")), Y_val[i]])

In [8]:
train_loader = DataLoader(prep_training, batch_size=16, shuffle=True)
valid_loader = DataLoader(prep_val, batch_size=16, shuffle=False)
test_loader = DataLoader(prep_test, batch_size=16, shuffle=False)

In [9]:
def CheckAccuracy(predictions, labels):
        acc = 0.0
        for i in range(len(predictions)):
            if (predictions[i] == labels[i]):
                acc += 1
        return acc/len(predictions)

In [10]:

epochs = 10
lr = 0.00001
loss_function = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

cuda = True # Set this if training on GPU
cuda = cuda and torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

In [11]:

# [batch_size, seq_len, input_size]
class LSTM_Net(nn.Module):
    
    def __init__(self):
        super(LSTM_Net, self).__init__()
        self.lstm = nn.LSTM(input_size=768, hidden_size=768, num_layers=1, batch_first=True) #bidirectional=True
        self.fc = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
    

    def forward(self, x):
        print(x.shape)
        x = model(x)[0]
        #x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        output = self.fc(ht[-1, :, :])        
        output = self.sigmoid(output)
        return output


In [12]:
# Setting up model parameters
lstm_model = LSTM_Net().to(device)

early_stopping = 5
notImproved = 0
bestLoss = None
bestModel = None

trainArr = []
valArr = []

bestEpoch = 0

for epoch in range(1, epochs + 1): 
    
    train_loss = 0.0
    lstm_model.train()
    
    for batch_idx, data in enumerate(train_loader):
        
        # get the input
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        
        outputs = lstm_model(inputs)
        
        outputs = outputs.squeeze(1)
        
        loss = loss_function(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader.dataset)
    
    #print("train loss: ", train_loss)
    
    valid_loss = 0
    labs = []
    preds = []
    
    lstm_model.eval()
    with torch.no_grad():        
        for batch_idx, data in enumerate(valid_loader):
            # get the input
            inputs, labels = data
        
            inputs = inputs.to(device)
            labels = labels.to(device).float()
        
            # forward + backward + optimize
            outputs = lstm_model(inputs)
            outputs = outputs.squeeze(1)
        
            labs.extend(labels)
            preds.extend(torch.round(outputs))
            valid_loss += loss_function(outputs, labels).item()
            
    valid_loss /= len(valid_loader.dataset)
    
    
    #print("valid loss: ", valid_loss)
    
    print("Accuracy on validation set: ", CheckAccuracy(labs, preds))
    
    if bestLoss == None:
        bestLoss = valid_loss
    
    if valid_loss <= bestLoss:
        bestLoss = valid_loss
        bestModel = model
        notImproved = 0
        bestEpoch = epoch
    else:
        notImproved +=1
        
    if notImproved >= early_stopping:
        break

torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 100])


KeyboardInterrupt: 

In [17]:
lstm_model.eval()
with torch.no_grad():        
    for batch_idx, data in enumerate(test_loader):
        # get the input
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        
        # forward + backward + optimize
        outputs = lstm_model(inputs)
        outputs = outputs.squeeze(1)
        
        labs.extend(labels)
        preds.extend(torch.round(outputs))
print("Accuracy on test set: ", CheckAccuracy(labs, preds))

Accuracy on test set:  0.6798611111111111
